In [ ]:
import json
import os
import requests
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import datashade
import hvplot.xarray
import pytest
import xarray as xr
hv.extension('bokeh', width=100)

## 1. API 

Das [Docker Repo](https://hub.docker.com/r/felixgi1516/geosoft2_frontend/) enthält jetzt ein funktionierendes Image des API Repos auf Github.

#### Anlegen des Docker Containers:

Image herunterladen und als Container starten:

Mit dem folgenden Befehl kann man das Image herunterladen und starten:
```bash
docker run --name fe_server_ii -p 80:80 felixgi1516/geosoft2_frontend:latest
```

Dabei entsteht ein Docker Container mit Namen `fe_server_ii` welches auf den Port 80 hört.

In [ ]:
#Starten des Containers
! docker start fe_server_ii

Die Beschreibungen der Prozesse sind keine Platzhalter mehr, sondern sind auf unsere Prozesse angepasst. 

In [ ]:
#Load in Processes Response JSON
z = requests.get('http://localhost/api/v1/processes')
processes  = 'Processes:\n \n'+ json.dumps(z.json(),indent=1) + '\n \n'

print(processes)

In [ ]:
#Stoppen des Servers
! docker stop fe_server_ii

## 2. Datacubes

#### 2.1. Sentinel Datacubes

Der automaisierte Download von Sentinel SAFE Dateien funktoioniert ohne Weiteres. Da der Download aufgrund der großen Dateien aber etwas dauert, wird er hier nicht gezeigt. 

Auch Umwandlung und Mergen der SAFE Dateien ist zeitaufwendig, deshalb wird es nur an dieser Stelle nur besprochen. 

So sieht der Aufruf für den folgenden Beispiel Datacube aus:  
```
directory = 'C:/GIS/demodata/'
resolution = 100    #10, 20, 60, 100 möglich

'''Parameters for the download'''
#BBOX für Münster
aoi = 'POLYGON((7.52834379254901 52.01238155392252,7.71417925515199 52.01183230436206,7.705255583805303 51.9153349236737,7.521204845259327 51.90983021961716,7.52834379254901 52.01238155392252,7.52834379254901 52.01238155392252))'
collectionDate = ('20200601', '20200610')
clouds = (0,30)

main( resolution, directory, collectionDate, aoi, clouds, username, password, True, False)
```

Die Auswahl der umzuwandelden Bilder kann über **Auflösung** (10 x 10m, 20 x 20m, 30 x 30m, 100 x 100m), **Area of Interest** (Als Polygon mit Eckkoordinaten), **Aufnahmezeitraum** und **Wolkenbedeckung**.

Der Beispieldatensatz des Sentinel Tiles [32ULC](https://maps.eatlas.org.au/index.html?intro=false&z=7&ll=8,52&l0=ea_ref%3AWorld_ESA_Sentinel-2-tiling-grid_Poly,ea_ea-be%3AWorld_Bright-Earth-e-Atlas-basemap,google_HYBRID,google_TERRAIN,google_SATELLITE,google_ROADMAP&v0=,,f,f,f,f) im Februar 2020:

In [ ]:
print(os.listdir('../demodata'))
dc = xr.open_dataset('../demodata/datacube_2020-06-01_T32ULC_R100.nc')
dc

##### Der rote Kanal des Datensatzes

In [ ]:
dc.red.isel(time=0).hvplot(cmap="Gray")

##### Der Nahinfrarotkanal des Datensatzes 

In [ ]:
dc.nir.isel(time=0).hvplot(cmap="Gray")

#### 2.2 SST Datacubes
Die SST Datencubes können auch heruntergeladen,gemergt und gespeichert werden.

Eine Konversion der Daten ist nicht nötig, da es schon netCDF Daten sind. 

##### Ein gemergter Datacube für die Jahre 2018 und 2019:

In [ ]:
ds_2018_2019 = xr.open_dataset("../demodata/sst.day.mean.2018_2019.nc")

In [ ]:
ds_2018_2019.sst.hvplot(groupby='time', cmap="coolwarm")

## 3. NDVI

Mit den neuen Datacubes aus den Sentineldaten ist es jetzt möglich den NDVI zu berechnen.

In [ ]:
%run ndvi

##### Der NDVI für die Region aus dem Beispiel Sentinel Datacube:

In [ ]:
red_nir_ulc = prepareData('..\demodata\datacube_2020-06-01_T32ULC_R100.nc')
ndvi_ulc = calculate(red_nir_ulc[0], red_nir_ulc[1])
ndvi_ulc.hvplot(cmap='BrBG')

##### Münster im Zoom:

In [ ]:
ndvi_ulc[380:480,1000:1098].hvplot(cmap='BrBG')

##### Der NDVI für das gleiche Tile im Februar 2020:

In [ ]:
red_nir_feb = prepareData('..\demodata\datacube_2020-02-07_T32ULC_R100.nc')
calculate_with_dask(red_nir_feb[0], red_nir_feb[1], '..\demodata\datacube_2020-02-07_T32ULC_R100_NDVI.nc')
ndvi_feb =  xr.open_dataset("..\demodata\datacube_2020-02-07_T32ULC_R100_NDVI.nc")
ndvi_feb.hvplot(cmap='BrBG')

##### Münster im Februar

In [ ]:
ndvi_feb.__xarray_dataarray_variable__[380:480,1000:1098].hvplot(cmap='BrBG')

## 4. SST

In [ ]:
%run mean_sst

 ##### Berechnung des Standardwerts für den Beispiel Datnsatz von oben:

In [ ]:
ds = xr.open_dataset("../demodata/sst.day.mean.2018_2019.nc", chunks={"time": "auto"})

x = wrapper_mean_sst(ds, ['2018-01-01','2019-12-31'], [0,30,50,80])

x = xr.open_dataset(x)

# visualisieren

sst_x = hv.Dataset(x, kdims=['lon', 'lat'])
sst_x = sst_x.to(hv.QuadMesh, kdims=["lon", "lat"])
get_ipython().run_line_magic('opts', 'RGB [width=900 height=600]')
datashade(sst_x, precompute=True, cmap=plt.cm.RdBu_r)

#### 4.2 Exceptions

Die SST Methode wirft jetzt Exceptions bei unvollstäniger Parameterangabe oder Logikfehlern.

In [ ]:
# Unvollständige Parameter
wrapper_mean_sst(ds,[0.1])
wrapper_mean_sst(ds, ['2018-01-01','2020-12-31'], [0,30,50])

# Logikfehler z.B. Längengrad > 90°
wrapper_mean_sst(ds, ['2018-01-01','2020-12-31'], [180,30,50,80])
wrapper_mean_sst(ds, ['2018-01-01','2020-12-31'], [0,30,50,80])

## 5.  Testen
Sowohl für den NDVI und SST gibt es jetzt Unittests. Unten wird lassen wir einmal Pytest laufen, um die Ergebnisse zu demonstrieren. 

In [ ]:
! pytest